<a href="https://colab.research.google.com/github/smddkt/NLP_Project/blob/main/Text_Processing_Lab_Week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## N-gram

In [2]:
#sample sentence

s = "one two three four five"

tokens = s.split(" ")
print(tokens)

['one', 'two', 'three', 'four', 'five']


In [3]:
sequences = [tokens[i:] for i in range(2)]
# i = 1일 때, i = 2일 때. i부터 끝까지 토큰을 출력하는 시퀀스

In [4]:
sequences

[['one', 'two', 'three', 'four', 'five'], ['two', 'three', 'four', 'five']]

In [5]:
bigrams = zip(*sequences)

#zip function: 여러 개의 시퀀스(예: 리스트, 튜플 등)를 인자로 받아 각 시퀀스에서 하나의 요소를 차례대로 가져와서 튜플로 묶어주는 역할

In [6]:
print(list(bigrams))
#list(zip(*sequences))는 sequences 리스트에 있는 각 시퀀스에서 동일한 인덱스에 있는 요소들을 묶어서 튜플로 만든 후, 그 결과를 리스트로 변환한다.

[('one', 'two'), ('two', 'three'), ('three', 'four'), ('four', 'five')]


**trigram**

In [7]:
sequences = [tokens[i:] for i in range(3)]
sequences

[['one', 'two', 'three', 'four', 'five'],
 ['two', 'three', 'four', 'five'],
 ['three', 'four', 'five']]

In [8]:
trigrams = zip(*sequences)
print(list(trigrams))
#  zip 함수는 자동으로 n-gram 생성을 종료할 수 있음. 세 번째만에 멈췄는데,  마지막 시퀀스의 원소가 세 개밖에 없기 때문이다.

[('one', 'two', 'three'), ('two', 'three', 'four'), ('three', 'four', 'five')]


## n-grams를 생성하는 함수 만들기


(위의 sample_input 이용)

In [9]:
import re

def generate_ngrams(s, n):
  global sample_input
  #convert to lowercases
  sample_input = sample_input.lower()
  #replace all none alphanumeric characters with spaces
  sample_input = re.sub(r'[^a-zA-Z0-9]', ' ', sample_input)

  #break sentence in the token, remove empty tokens
  tokens = [token for token in s.split(" ") if token != ""]

  #use the zip function to help us generate n-grams
  #Concentrate the tokens into ngrams and return
  ngrams = zip(*[tokens[i:] for i in range(n)])

  return [" ".join(ngram) for ngram in ngrams]

### sample_input = re.sub(r'[^a-zA-Z0-9]', ' ', sample_input)


여기서 사용된 re.sub() 함수는 문자열에서 특정 패턴을 다른 문자열로 대체하는 역할을 합니다.

이 함수의 인자는 다음과 같습니다:

r'[^a-zA-Z0-9]': 정규 표현식 패턴입니다. 이 패턴은 문자열에서 알파벳 (대문자 및 소문자)과 숫자를 제외한 모든 문자를 찾습니다. 여기서 ^는 부정을 나타내므로, 대괄호 안에 있는 문자들을 제외한 모든 문자를 의미합니다.
' ': 대체할 문자열입니다. 여기서는 정규 표현식 패턴과 매칭되는 문자열을 공백 문자로 대체합니다.
sample_input: 대상 문자열입니다. 정규 표현식 패턴에 따라 대체할 대상이 됩니다.
따라서 이 코드는 sample_input 문자열에서 알파벳 (대문자 및 소문자)과 숫자를 제외한 모든 문자를 공백으로 대체하여 반환합니다.

In [10]:
sample_input = """Natural-language processing (NLP) is an area of
computer science and artificial intelligence
concerned with the interactions between computers
and human (natural) languages."""

In [11]:
generate_ngrams(sample_input, n=5)

['Natural-language processing (NLP) is an',
 'processing (NLP) is an area',
 '(NLP) is an area of\ncomputer',
 'is an area of\ncomputer science',
 'an area of\ncomputer science and',
 'area of\ncomputer science and artificial',
 'of\ncomputer science and artificial intelligence\nconcerned',
 'science and artificial intelligence\nconcerned with',
 'and artificial intelligence\nconcerned with the',
 'artificial intelligence\nconcerned with the interactions',
 'intelligence\nconcerned with the interactions between',
 'with the interactions between computers\nand',
 'the interactions between computers\nand human',
 'interactions between computers\nand human (natural)',
 'between computers\nand human (natural) languages.']

## N-gram language models using NLTK

In [12]:
#텍스트는 문장으로, 문장은, 문자열의 리스트로 이루어져 있다. 만약 텍스트 자체를 단순히 문자들의 집합으로 본다면?

text = [['a', 'b', 'c'], ['a', 'c', 'd', 'c', 'e', 'f']]

In [13]:
#이 텍스트를 bigram 모델로 만들기. NLTK의 함수를 이용하자.

from nltk.util import bigrams

list(bigrams(text[0]))

[('a', 'b'), ('b', 'c')]

**padding 추가하기**

위에서 만든 bigram 모델의 경우, 문장의 시작과 끝인 a와 c는 의미상 중요할 가능성이 큰데, 다른 단어들과의 연결은 상대적으로 적다. b는 두 번 나올 동안 a, c는 한 번 씩만 나옴.

Padding은 문장의 시작과 끝을 나타내는 특별한 기호를 추가하여 이러한 문제를 해결하는 방법이다. 이렇게 함으로써 문장의 시작과 끝에 있는 단어들도 주변 단어들과의 관계를 가지게 된다.


--->문맥 이해가 더 용이해진다.

In [14]:
from nltk.util import pad_sequence
list(pad_sequence(text[0],
                  pad_left = True,
                  left_pad_symbol = "<s>",
                  pad_right = True,
                  right_pad_symbol="</s>",
                  n=2))

['<s>', 'a', 'b', 'c', '</s>']

In [15]:
list(pad_sequence(text[0],
                  pad_left = True,
                  left_pad_symbol= "<s>",
                  pad_right=True,
                  right_pad_symbol="</s>",
                  n=3))

['<s>', '<s>', 'a', 'b', 'c', '</s>', '</s>']

pad_sequence 사용하면 자동으로 패딩 개수 맞춰짐. bigram이면 한 개의 패딩, trigram이면 두 개의 패딩...

근데 이 모든 변수를 매번 쓰기도 귀찮다. 거의 모든 경우에 이 변수들은 그냥 기본값으로 맞춰놓고 쓰기 때문에. 그래서 이제부터는 pad_sequence와 같은 변수들을 기본값으로 세팅해 놓은 함수인


pad_both_ends를 쓰면 된다.

In [16]:
from nltk.lm.preprocessing import pad_both_ends
list(pad_both_ends(text[0], n=2))

['<s>', 'a', 'b', 'c', '</s>']

In [17]:
#지금까지 한 것 합치기

list(bigrams(pad_both_ends(text[0], n=2)))

[('<s>', 'a'), ('a', 'b'), ('b', 'c'), ('c', '</s>')]

In [18]:
#"everygrams" 함수는 텍스트에서 가능한 모든 n-grams를 생성하는 데에 사용되며, 이해하기 쉬운 방식으로 구현되어 있지만, 대규모 데이터나 긴 텍스트에 적용할 때에는 처리 속도가 느릴 수 있다.

from nltk.util import everygrams
padded_bigrams = list(pad_both_ends(text[0], n=2))
list(everygrams(padded_bigrams, max_len=2))

[('<s>',),
 ('<s>', 'a'),
 ('a',),
 ('a', 'b'),
 ('b',),
 ('b', 'c'),
 ('c',),
 ('c', '</s>'),
 ('</s>',)]

In [19]:
padded_trigrams = list(pad_both_ends(text[0], n=3))
list(everygrams(padded_trigrams, max_len=3))

[('<s>',),
 ('<s>', '<s>'),
 ('<s>', '<s>', 'a'),
 ('<s>',),
 ('<s>', 'a'),
 ('<s>', 'a', 'b'),
 ('a',),
 ('a', 'b'),
 ('a', 'b', 'c'),
 ('b',),
 ('b', 'c'),
 ('b', 'c', '</s>'),
 ('c',),
 ('c', '</s>'),
 ('c', '</s>', '</s>'),
 ('</s>',),
 ('</s>', '</s>'),
 ('</s>',)]

In [20]:
from nltk.lm.preprocessing import flatten
list(flatten(pad_both_ends(sent, n=2) for sent in text))

['<s>', 'a', 'b', 'c', '</s>', '<s>', 'a', 'c', 'd', 'c', 'e', 'f', '</s>']

In [21]:
#위의 결과를 flat stream으로 만들기

from nltk.lm.preprocessing import flatten
list(flatten(pad_both_ends(sent, n=2) for sent in text))

['<s>', 'a', 'b', 'c', '</s>', '<s>', 'a', 'c', 'd', 'c', 'e', 'f', '</s>']

In [22]:
#in most cases we want to use the same text as the source for both vocabulary/n-gram counts.
# padded_everygram_pipeline 이라는 함수로 실현할 수 있다.

from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(2, text)

In [23]:
training_ngrams, padded_sentences = padded_everygram_pipeline(2, text)
for ngramlize_sent in training_ngrams:
  print(list(ngramlize_sent))

print(list(padded_sentences))

[('<s>',), ('<s>', 'a'), ('a',), ('a', 'b'), ('b',), ('b', 'c'), ('c',), ('c', '</s>'), ('</s>',)]
[('<s>',), ('<s>', 'a'), ('a',), ('a', 'c'), ('c',), ('c', 'd'), ('d',), ('d', 'c'), ('c',), ('c', 'e'), ('e',), ('e', 'f'), ('f',), ('f', '</s>'), ('</s>',)]
['<s>', 'a', 'b', 'c', '</s>', '<s>', 'a', 'c', 'd', 'c', 'e', 'f', '</s>']


## **Training**

In [24]:
from nltk.lm import MLE
lm = MLE(2)

#MLE = Maximum Likelihood Estimater.

In [ ]:
len(lm.vocab)
#2가 나온다.

In [25]:
lm.fit(train, vocab)
print(lm.vocab)


<Vocabulary with cutoff=1 unk_label='<UNK>' and 9 items>


In [26]:
len(lm.vocab)
#이제 9가 나온다.

9

In [27]:
list(lm.vocab)

['<s>', 'a', 'b', 'c', '</s>', 'd', 'e', 'f', '<UNK>']

## **trained model 사용하기**

In [28]:
print(lm.counts)

<NgramCounter with 2 ngram orders and 24 ngrams>


In [29]:
lm.counts['a']

2

In [30]:
lm.counts[['a']]

FreqDist({'b': 1, 'c': 1})

In [31]:
lm.counts[['a']]['b']

1

In [32]:
lm.score('a')

0.15384615384615385

In [33]:
lm.score('b', ['a'])

0.5

In [34]:
lm.logscore("a")

-2.700439718141092

In [35]:
test = [('a', 'b'), ('c', 'd')]
lm.entropy(test)

1.292481250360578

In [36]:
lm.perplexity(test)

2.449489742783178

In [37]:
lm.generate(1, random_seed=3)

'<s>'

In [38]:
lm.generate(5, random_seed=3)

['<s>', 'a', 'b', 'c', 'd']

In [39]:
#text seed로 조건 주기

lm.generate(5, text_seed = ['c'], random_seed=3)

['</s>', 'c', 'd', 'c', 'd']